# Setting Up the Environment

In [1]:
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# install nnunetv2
!pip install nnunetv2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 kB 5.8 MB/s eta 0:00:00a 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 7.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 3.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 91.9 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 65.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 23.1 MB/s eta 0:00:0000:01
 

In [2]:
# check nnunetv2
import nnunetv2

In [3]:
# Make Directory to store Models
!mkdir -p /kaggle/working/nnUNet_models/Dataset500_KiTS2023

In [4]:
# Make Directory to align with nnUNet
!mkdir -p /kaggle/working/nnUNet_raw/Dataset500_KiTS2023/

In [5]:
# Make Directory to align with nnUNet
!mkdir -p /kaggle/working/nnUNet_preprocessed

In [6]:
import os

# Raw data (can be empty if you only do inference)
os.environ["nnUNet_raw"] = "/kaggle/working/nnUNet_raw"

# Preprocessed data (can be empty if you only do inference)
os.environ["nnUNet_preprocessed"] = "/kaggle/working/nnUNet_preprocessed"

# Model results folder (where your Dataset700_MYTASK lives)
os.environ["nnUNet_results"] = "/kaggle/working/nnUNet_models"


In [7]:
# #Code to remove file
# import os

# file_path = "/kaggle/working/nnUNet_raw/Dataset500_KiTS2023/imaging_001_0000.nii.gz"

# if os.path.exists(file_path):
#     os.remove(file_path)
#     print("File deleted:", file_path)
# else:
#     print("File not found:", file_path)


In [8]:
# # Code to remove folder
# import shutil

# folder_path = "/kaggle/working/nnUNet_preprocessed/Dataset500_KiTS2023"

# # Remove the folder and everything inside
# shutil.rmtree(folder_path)


# Data Loading & Pre-Processing

In [9]:
import os
import SimpleITK as sitk

# === Directories ===
input_base = '/kaggle/input/nnunet-training/first50/imagesTr'
output_base = '/kaggle/working/nnUNet_raw/Dataset500_KiTS2023/imagesTr'
os.makedirs(output_base, exist_ok=True)

# === Range of cases to convert ===
for i in range(0, 40):  
    case_id = f"imaging_{i:03d}_0000.nii"
    input_folder = os.path.join(input_base, case_id)
    input_file = os.path.join(input_folder, "imaging.nii")
    output_file = os.path.join(output_base, f"{case_id}.gz")  # final output: imaging_000_0000.nii.gz
    
    print(f"\n Processing: {input_file}")
    
    try:
        # Read input .nii
        img = sitk.ReadImage(input_file)
        
        # Write compressed .nii.gz
        writer = sitk.ImageFileWriter()
        writer.SetFileName(output_file)
        writer.UseCompressionOn()
        writer.Execute(img)
        
        # Show success message
        print(f" Saved compressed file: {output_file}")
        
        # Optional: check size
        original_size = os.path.getsize(input_file) / (1024 * 1024)
        compressed_size = os.path.getsize(output_file) / (1024 * 1024)
        print(f"   Original: {original_size:.2f} MB | Compressed: {compressed_size:.2f} MB")
        
    except Exception as e:
        print(f" Error processing {input_file}: {e}")


 Processing: /kaggle/input/nnunet-training/first50/imagesTr/imaging_000_0000.nii/imaging.nii
 Saved compressed file: /kaggle/working/nnUNet_raw/Dataset500_KiTS2023/imagesTr/imaging_000_0000.nii.gz
   Original: 1222.00 MB | Compressed: 158.84 MB

 Processing: /kaggle/input/nnunet-training/first50/imagesTr/imaging_001_0000.nii/imaging.nii
 Saved compressed file: /kaggle/working/nnUNet_raw/Dataset500_KiTS2023/imagesTr/imaging_001_0000.nii.gz
   Original: 1204.00 MB | Compressed: 199.54 MB

 Processing: /kaggle/input/nnunet-training/first50/imagesTr/imaging_002_0000.nii/imaging.nii
 Saved compressed file: /kaggle/working/nnUNet_raw/Dataset500_KiTS2023/imagesTr/imaging_002_0000.nii.gz
   Original: 522.00 MB | Compressed: 72.11 MB

 Processing: /kaggle/input/nnunet-training/first50/imagesTr/imaging_003_0000.nii/imaging.nii
 Saved compressed file: /kaggle/working/nnUNet_raw/Dataset500_KiTS2023/imagesTr/imaging_003_0000.nii.gz
   Original: 540.00 MB | Compressed: 86.36 MB

 Processing: /kaggl

In [10]:
import os
import SimpleITK as sitk

# === Directories ===
input_base = '/kaggle/input/nnunet-training/first50/labelsTr'
output_base = '/kaggle/working/nnUNet_raw/Dataset500_KiTS2023/labelsTr'
os.makedirs(output_base, exist_ok=True)

# === Range of cases to convert ===
for i in range(0, 40):  
    case_id = f"imaging_{i:03d}.nii"
    input_folder = os.path.join(input_base, f"imaging_{i:03d}.nii")
    input_file = os.path.join(input_folder, "segmentation.nii")
    output_file = os.path.join(output_base, f"{case_id}.gz")  # final output: imaging_000_0000.nii.gz
    
    print(f"\n Processing: {input_file}")
    
    try:
        # Read input .nii
        img = sitk.ReadImage(input_file)
        
        # Write compressed .nii.gz
        writer = sitk.ImageFileWriter()
        writer.SetFileName(output_file)
        writer.UseCompressionOn()
        writer.Execute(img)
        
        # Show success message
        print(f" Saved compressed file: {output_file}")
        
        # Optional: check size
        original_size = os.path.getsize(input_file) / (1024 * 1024)
        compressed_size = os.path.getsize(output_file) / (1024 * 1024)
        print(f"   Original: {original_size:.2f} MB | Compressed: {compressed_size:.2f} MB")
        
    except Exception as e:
        print(f" Error processing {input_file}: {e}")


 Processing: /kaggle/input/nnunet-training/first50/labelsTr/imaging_000.nii/segmentation.nii
 Saved compressed file: /kaggle/working/nnUNet_raw/Dataset500_KiTS2023/labelsTr/imaging_000.nii.gz
   Original: 152.75 MB | Compressed: 0.19 MB

 Processing: /kaggle/input/nnunet-training/first50/labelsTr/imaging_001.nii/segmentation.nii
 Saved compressed file: /kaggle/working/nnUNet_raw/Dataset500_KiTS2023/labelsTr/imaging_001.nii.gz
   Original: 150.50 MB | Compressed: 0.23 MB

 Processing: /kaggle/input/nnunet-training/first50/labelsTr/imaging_002.nii/segmentation.nii
 Saved compressed file: /kaggle/working/nnUNet_raw/Dataset500_KiTS2023/labelsTr/imaging_002.nii.gz
   Original: 65.25 MB | Compressed: 0.10 MB

 Processing: /kaggle/input/nnunet-training/first50/labelsTr/imaging_003.nii/segmentation.nii
 Saved compressed file: /kaggle/working/nnUNet_raw/Dataset500_KiTS2023/labelsTr/imaging_003.nii.gz
   Original: 67.50 MB | Compressed: 0.10 MB

 Processing: /kaggle/input/nnunet-training/first5

In [11]:
!cp /kaggle/input/nnunet-training/first50/dataset.json /kaggle/working/nnUNet_raw/Dataset500_KiTS2023

In [12]:
# Pre processing with reduced workload
import os

os.environ["OMP_NUM_THREADS"] = "1"
os.environ["nnUNet_tmp"] = "/content/tmp"
os.makedirs("/content/tmp", exist_ok=True)

In [13]:
# Data Pre Processing with reduce Workload
!nnUNetv2_plan_and_preprocess -d 500 -c 3d_fullres --verify_dataset_integrity --verbose -np 2 -pl nnUNetPlannerResEncM

Fingerprint extraction...
Dataset500_KiTS2023
Using <class 'nnunetv2.imageio.simpleitk_reader_writer.SimpleITKIO'> as reader/writer

####################
verify_dataset_integrity Done. 
If you didn't see any error messages then your dataset is most likely OK!
####################

Using <class 'nnunetv2.imageio.simpleitk_reader_writer.SimpleITKIO'> as reader/writer
Experiment planning...
Attempting to find 3d_lowres config. 
Current spacing: [0.83286186 0.83285156 0.83285156]. 
Current patch size: (128, 128, 128). 
Current median shape: [490.77669903 497.08737864 497.08737864]
Attempting to find 3d_lowres config. 
Current spacing: [0.85784772 0.85783711 0.85783711]. 
Current patch size: (128, 128, 128). 
Current median shape: [476.48223207 482.60910548 482.60910548]
Attempting to find 3d_lowres config. 
Current spacing: [0.88358315 0.88357222 0.88357222]. 
Current patch size: (128, 128, 128). 
Current median shape: [462.6041088  468.55252959 468.55252959]
Attempting to find 3d_lowres c

# Model Training

In [ ]:
!nnUNetv2_train Dataset500_KiTS2023 3d_fullres all -p nnUNetResEncUNetMPlans -tr nnUNetTrainer_10epochs

/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:2225: UnsupportedFieldAttributeWarning: The 'repr' attribute with value False was provided to the `Field()` function, which has no effect in the context it was used. 'repr' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` statement was used, or if the `Field()` function was attached to a single member of a union type.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:2225: UnsupportedFieldAttributeWarning: The 'frozen' attribute with value True was provided to the `Field()` function, which has no effect in the context it was used. 'frozen' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` 

In [ ]:
# import nibabel as nib
# import numpy as np
# import matplotlib.pyplot as plt
# from matplotlib.widgets import Slider
# from matplotlib.colors import ListedColormap
# from matplotlib.patches import Patch


# # Paths
# ct_path = "/kaggle/working/nnUNet_raw/Dataset500_KiTS2023/imaging_001_0000.nii.gz"
# seg_path = "/kaggle/working/results/imaging_001.nii.gz"

# # Load images
# ct_img = nib.load(ct_path)
# ct_data = ct_img.get_fdata()

# seg_img = nib.load(seg_path)
# seg_data = seg_img.get_fdata().astype(int)

# # --- 1. Custom Colors and Masking (Same as before) ---
# colors = ['#000000', '#9467BD', '#00BCD4', '#8BC34A']
# cmap = ListedColormap(colors)
# masked_seg_data = np.ma.masked_where(seg_data == 0, seg_data)
# cmap.set_bad(color='none', alpha=0.0)

# # --- 2. Function to Get Correct Axial Slice ---
# def get_axial_slice(data_3d, index, axis=0):
#     if axis == 0:
#         slice_2d = data_3d[index, :, :]
#     elif axis == 1:
#         slice_2d = data_3d[:, index, :]
#     else:
#         slice_2d = data_3d[:, :, index]
    
#     # FIX ROTATION: Use np.rot90 instead of .T to correct orientation
#     # k=1 rotates 90 degrees CCW, k=3 rotates 270 degrees CCW (90 degrees CW)
#     # This value is often found through trial and error (try k=1, 2, or 3)
#     return np.rot90(slice_2d, k=1) 

# def get_masked_axial_slice(data_3d, index, axis=0):
#     if axis == 0:
#         masked_slice = masked_seg_data[index, :, :]
#     elif axis == 1:
#         masked_slice = masked_seg_data[:, index, :]
#     else:
#         masked_slice = masked_seg_data[:, :, index]
        
#     # FIX ROTATION: Apply the same rotation
#     return np.rot90(masked_slice, k=1) 

# # --- 3. Initial Axial Slice Setup ---
# AXIAL_AXIS = 0 
# MAX_SLICES = ct_data.shape[AXIAL_AXIS]
# # FIX COLORS: Set initial slice to a value where structures are present (e.g., 150)
# slice_idx = 260

# # Plot setup
# fig, ax = plt.subplots(figsize=(6,6))
# plt.subplots_adjust(bottom=0.25)

# # Plot the corrected axial slice data
# img_plot = ax.imshow(get_axial_slice(ct_data, slice_idx, axis=AXIAL_AXIS), cmap='gray', vmin=-100, vmax=300)
# overlay = ax.imshow(get_masked_axial_slice(seg_data, slice_idx, axis=AXIAL_AXIS), cmap=cmap, alpha=0.5, interpolation='none')

# ax.set_title(f'Axial Slice {slice_idx} of {MAX_SLICES-1}')
# ax.axis('off')

# # --- 4. Slider and Update Function (Remains the same) ---
# ax_slider = plt.axes([0.25, 0.1, 0.5, 0.03])
# slider = Slider(ax_slider, 'Axial Slice', 0, MAX_SLICES-1, valinit=slice_idx, valstep=1)

# def update(val):
#     idx = int(slider.val)
#     img_plot.set_data(get_axial_slice(ct_data, idx, axis=AXIAL_AXIS))
#     overlay.set_data(get_masked_axial_slice(seg_data, idx, axis=AXIAL_AXIS))
#     ax.set_title(f'Axial Slice {idx} of {MAX_SLICES-1}')
#     fig.canvas.draw_idle()

# slider.on_changed(update)

# # --- 5. Legend (Same as before) ---
# legend_elements = [
#     Patch(facecolor='#9467BD', alpha=0.5, label='Kidney (1)'),
#     Patch(facecolor='#00BCD4', alpha=0.5, label='Tumor (2)'),
#     Patch(facecolor='#8BC34A', alpha=0.5, label='Cyst (3)')
# ]
# ax.legend(handles=legend_elements, loc='lower right', bbox_to_anchor=(1.0, 0.0), framealpha=0.6)

# plt.show()

# Prediction & Scores

In [ ]:
# Dir. to store inferencing results
!mkdir -p /kaggle/working/inference_outs

In [ ]:
# # Run the Prediction 
# !nnUNetv2_predict -i /kaggle/working/nnUNet_raw/Dataset500_KiTS2023 -o /kaggle/working/results/ -d 500 -c 3d_fullres_batch_4_all -f all

In [ ]:
# kits23_compute_metrics FOLDER_WITH_PREDICTIONS -num_processes XX